# Data Processing For Fulton County Sales and Parcel Data

In [1]:
import pandas as pd
import os
import polars as pl

pd.set_option('display.max_columns', 200)

In [16]:
#def clean_sales(df):
#def clean_parcel(df):

def read_file(file, library):
    print("Reading file: ", file)
    df = pd.DataFrame()
    if library == 'Polars':
        if file[-4:len(file)] == "xlsx":
            df = pl.read_excel(
                '../data/digest/' + file,
            xlsx2csv_options={"ignore_formats":["date", "float"], "ignore_errors":"True", "infer_schema_length":0},
            read_csv_options={"infer_schema_length": 0, "dtypes": {"Taxyr": pl.Utf8}}
            #usecols=vars,
            #dtype=datatypes
            )
        else:
            df = pl.read_csv(
            '../data/sales/'+ file,
            separator='\t',
            ignore_errors=True,
            dtypes={"Taxyr": pl.Utf8}
            #dtype=datatypes
            )
    else:
        if file[-4:len(file)] == "xlsx":
            df = pd.read_excel(
                '../data/digest/' + file,
                index_col=False,
            #usecols=vars,
            #dtype=datatypes
            )
        else:
            df = pd.read_csv(
                '../data/sales/'+ file,
                sep='\t',
            #dtype=datatypes
            )

    return df

In [3]:
path = "C:/Users/nicho/Documents/research/FCS/data/digest/"
parcel_files = os.listdir(path)

path = "C:/Users/nicho/Documents/research/FCS/data/sales/"
sales_files = os.listdir(path)

files_yr = {}

for yr in range(2011, 2022):
    for file in sales_files:
        if str(yr) in file:
            files_yr[yr] = [file]

    for file in parcel_files:
        if str(yr) in file:
            files_yr[yr].append(file)

In [4]:
files_yr

{2011: ['Sales2011.txt',
  'parcel_14_2010-2011.xlsx',
  'parcel_17_2010-2011.xlsx',
  'parcel_atl_2011-2012.xlsx',
  'parcel_nf_2010-2011.xlsx',
  'parcel_sf_2010-2011.xlsx'],
 2012: ['Sales2012.txt',
  'parcel_14_2012-2013.xlsx',
  'parcel_17_2012-2013.xlsx',
  'parcel_atl_2011-2012.xlsx',
  'parcel_nf_2012-2013.xlsx',
  'parcel_sf_2012-2013.xlsx'],
 2013: ['Sales2013.txt',
  'parcel_14_2012-2013.xlsx',
  'parcel_17_2012-2013.xlsx',
  'parcel_atl_2013-2014.xlsx',
  'parcel_nf_2012-2013.xlsx',
  'parcel_sf_2012-2013.xlsx'],
 2014: ['Sales2014.txt',
  'parcel_14_2014.xlsx',
  'parcel_17_2014.xlsx',
  'parcel_atl_2013-2014.xlsx',
  'parcel_nf_2014.xlsx',
  'parcel_sf_2014.xlsx'],
 2015: ['Sales2015.txt',
  'parcel_14_2015.xlsx',
  'parcel_17_2015.xlsx',
  'parcel_atl_2015.xlsx',
  'parcel_nf_2015.xlsx',
  'parcel_sf_2015.xlsx'],
 2016: ['Sales2016.txt',
  'parcel_17_2016.xlsx',
  'parcel_atl_2016.xlsx',
  'parcel_nf_2016.xlsx',
  'parcel_sf_2016.xlsx'],
 2017: ['Sales2017.txt',
  'parce

In [280]:
yr_dfs = []
sum_additional_rows = 0
total_own_null = 0

for yr_files in files_yr.values():
    sales_df = None
    parcel_dfs = []

    for filename in yr_files:
        df = read_file(filename, 'Polars')
        if 'parcel' in filename:
            #df = clean_parcel(df)
            parcel_dfs.append(df)
        else:
            #df = clean_sales(df)
            sales_df = df

    init_len = sales_df.select(pl.count()).item()
    print("Length of original sales: ", init_len)

    parcels_appended = pl.concat(parcel_dfs)
    sales_df = sales_df.select(
        pl.col("Taxyr").cast(pl.Int16, strict=False),
        pl.col("Parid").cast(pl.Utf8, strict=False),
        pl.col("*").exclude("Taxyr", "Parid")
    ).with_columns(
        (pl.col("Parid").str.replace_all(' ', '')).alias("Parid_strip")
    )
    parcels_appended = parcels_appended.select(
        pl.col("Taxyr").cast(pl.Int16, strict=False),
        pl.col("Parid").cast(pl.Utf8, strict=False),
        pl.col("*").exclude("Taxyr", "Parid")
    ).with_columns(
        (pl.col("Parid").str.replace_all(' ', '')).alias("Parid_strip")
    )
    null_count = len(parcels_appended.filter(
        pl.col('Own1').is_null()
    ))
    print("Total Null: ", null_count)
    total_own_null = null_count
    combined = sales_df.join(parcels_appended, how="left", on=['Parid_strip', 'Taxyr'])
    print("Successfully merged above files")
    print("Additional rows generated (merging issue): ", init_len - combined.select(pl.count()).item())
    sum_additional_rows = sum_additional_rows + (init_len - combined.select(pl.count()).item())
    yr_dfs.append(combined)

Reading file:  Sales2011.txt
Reading file:  parcel_14_2010-2011.xlsx
Reading file:  parcel_17_2010-2011.xlsx
Reading file:  parcel_atl_2011-2012.xlsx
Reading file:  parcel_nf_2010-2011.xlsx
Reading file:  parcel_sf_2010-2011.xlsx
Length of original sales:  47806
Total Null:  13
Successfully merged above files
Additional rows generated (merging issue):  -20596
Reading file:  Sales2012.txt
Reading file:  parcel_14_2012-2013.xlsx
Reading file:  parcel_17_2012-2013.xlsx
Reading file:  parcel_atl_2011-2012.xlsx
Reading file:  parcel_nf_2012-2013.xlsx
Reading file:  parcel_sf_2012-2013.xlsx


Exception ignored in: <function Xlsx2csv.__del__ at 0x000001D82809C720>
Traceback (most recent call last):
  File "c:\Users\nicho\anaconda3\envs\fcs\Lib\site-packages\xlsx2csv.py", line 219, in __del__
    self.ziphandle.close()
    ^^^^^^^^^^^^^^
AttributeError: 'Xlsx2csv' object has no attribute 'ziphandle'
Exception ignored in: <function Xlsx2csv.__del__ at 0x000001D82809C720>
Traceback (most recent call last):
  File "c:\Users\nicho\anaconda3\envs\fcs\Lib\site-packages\xlsx2csv.py", line 219, in __del__
    self.ziphandle.close()
    ^^^^^^^^^^^^^^
AttributeError: 'Xlsx2csv' object has no attribute 'ziphandle'


Length of original sales:  39987
Total Null:  13
Successfully merged above files
Additional rows generated (merging issue):  -17247
Reading file:  Sales2013.txt
Reading file:  parcel_14_2012-2013.xlsx
Reading file:  parcel_17_2012-2013.xlsx
Reading file:  parcel_atl_2013-2014.xlsx
Reading file:  parcel_nf_2012-2013.xlsx
Reading file:  parcel_sf_2012-2013.xlsx
Length of original sales:  47461
Total Null:  13
Successfully merged above files
Additional rows generated (merging issue):  -18921
Reading file:  Sales2014.txt
Reading file:  parcel_14_2014.xlsx
Reading file:  parcel_17_2014.xlsx
Reading file:  parcel_atl_2013-2014.xlsx
Reading file:  parcel_nf_2014.xlsx
Reading file:  parcel_sf_2014.xlsx
Length of original sales:  18692
Total Null:  8
Successfully merged above files
Additional rows generated (merging issue):  -4525
Reading file:  Sales2015.txt
Reading file:  parcel_14_2015.xlsx
Reading file:  parcel_17_2015.xlsx
Reading file:  parcel_atl_2015.xlsx
Reading file:  parcel_nf_2015.x

In [281]:
cols = yr_dfs[0].columns
print(cols)

drop = {}
drop_var = []

for index, df in enumerate(yr_dfs):
    print(df.shape)
    for col in df.columns:
        if col not in cols and (col not in drop):
            if index not in drop.keys():
                drop[index] = [col]
            else:
                drop[index] = drop[index] + [col]
        if col not in cols and (col not in drop_var):
            drop_var = drop_var + [col]
print(cols)
print(drop)

['Taxyr', 'Parid', 'Saledt: Year (YYYY)', 'Saledt: Month (Mon)', 'Taxdist', 'Nbhd', 'Class', 'Luc', 'Saledt', 'Book', 'Page', 'SALES PRICE', 'FAIR MARKET VALUE', 'DEED TYPE', 'Aprland', 'Aprbldg', 'Costval', 'Saleval', 'Who', 'Wen', 'GRANTOR', 'GRANTEE', 'Revcode', 'Reascd', 'Adrpre', 'Adrno', 'Adrdir', 'Adrstr', 'Adrsuf', 'Adrsuf2', 'Cityname', 'Unitno', 'Livunit', 'Calcacres', 'Zoning', 'Notecd1', 'Notecd2', 'Chgrsn', 'Cur', 'Cur_duplicated_0', 'Parid_strip', 'Parid_right', 'Nbhd_right', 'Situs Adrno', 'Situs Adrdir', 'Situs Adrstr', 'Situs Adrsuf', 'Situs Adrsuf2', 'Cityname_right', 'Zoning_right', 'Muni', 'Class_right', 'Luc_right', 'Livunit_right', 'Calcacres_right', 'Location', 'Fronting', 'Street1', 'Street2', 'Traffic', 'Topo1', 'Topo2', 'Topo3', 'Util1', 'Util2', 'Util3', 'Parkprox', 'Parkquanit', 'Parktype', 'Note1', 'Note2', 'Note3', 'Note4', 'Notecd1_right', 'Notecd2_right', 'Ofcard', 'Bldgros D', 'Bldgros V', 'Mscbld N', 'Mscbld V', 'Chgrsn_right', 'Taxdist_right', 'Own1',

In [282]:
for index, df in enumerate(yr_dfs):
    if index in drop.keys():
        print(index, drop[index])
        yr_dfs[index] = yr_dfs[index].drop(drop[index])

2 ['Whocalc', 'Wencalc', 'Saletype']
3 ['Appraiser ID', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']
4 ['Appraiser ID', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']
5 ['Appraiser ID', 'Val30 SUM', 'Areasum', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']
6 ['Appraiser ID', 'Val30 SUM', 'Areasum', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']
7 ['Appraiser ID', 'Val30 SUM', 'Areasum', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']
8 ['Appraiser ID', 'Val30 SUM', 'Areasum', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']
9 ['Appraiser ID', 'Val30 SUM', 'Areasum', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']
10 ['Appraiser ID', 'Val30 SUM', 'Areasum', 'Income', 'Whocalc', 'Wencalc', 'Saletype', 'Bldgros V_right', 'Mscbld V_right']


In [283]:
cols = yr_dfs[0].columns
print(cols)

drop = {}
drop_var = []

for index, df in enumerate(yr_dfs):
    print(df.shape)
    for col in df.columns:
        if col not in cols and (col not in drop):
            if index not in drop.keys():
                drop[index] = [col]
            else:
                drop[index] = drop[index] + [col]
        if col not in cols and (col not in drop_var):
            drop_var = drop_var + [col]
print(cols)
print(drop)

['Taxyr', 'Parid', 'Saledt: Year (YYYY)', 'Saledt: Month (Mon)', 'Taxdist', 'Nbhd', 'Class', 'Luc', 'Saledt', 'Book', 'Page', 'SALES PRICE', 'FAIR MARKET VALUE', 'DEED TYPE', 'Aprland', 'Aprbldg', 'Costval', 'Saleval', 'Who', 'Wen', 'GRANTOR', 'GRANTEE', 'Revcode', 'Reascd', 'Adrpre', 'Adrno', 'Adrdir', 'Adrstr', 'Adrsuf', 'Adrsuf2', 'Cityname', 'Unitno', 'Livunit', 'Calcacres', 'Zoning', 'Notecd1', 'Notecd2', 'Chgrsn', 'Cur', 'Cur_duplicated_0', 'Parid_strip', 'Parid_right', 'Nbhd_right', 'Situs Adrno', 'Situs Adrdir', 'Situs Adrstr', 'Situs Adrsuf', 'Situs Adrsuf2', 'Cityname_right', 'Zoning_right', 'Muni', 'Class_right', 'Luc_right', 'Livunit_right', 'Calcacres_right', 'Location', 'Fronting', 'Street1', 'Street2', 'Traffic', 'Topo1', 'Topo2', 'Topo3', 'Util1', 'Util2', 'Util3', 'Parkprox', 'Parkquanit', 'Parktype', 'Note1', 'Note2', 'Note3', 'Note4', 'Notecd1_right', 'Notecd2_right', 'Ofcard', 'Bldgros D', 'Bldgros V', 'Mscbld N', 'Mscbld V', 'Chgrsn_right', 'Taxdist_right', 'Own1',

In [284]:
for index, df in enumerate(yr_dfs):
    print(yr_dfs[index].columns)

['Taxyr', 'Parid', 'Saledt: Year (YYYY)', 'Saledt: Month (Mon)', 'Taxdist', 'Nbhd', 'Class', 'Luc', 'Saledt', 'Book', 'Page', 'SALES PRICE', 'FAIR MARKET VALUE', 'DEED TYPE', 'Aprland', 'Aprbldg', 'Costval', 'Saleval', 'Who', 'Wen', 'GRANTOR', 'GRANTEE', 'Revcode', 'Reascd', 'Adrpre', 'Adrno', 'Adrdir', 'Adrstr', 'Adrsuf', 'Adrsuf2', 'Cityname', 'Unitno', 'Livunit', 'Calcacres', 'Zoning', 'Notecd1', 'Notecd2', 'Chgrsn', 'Cur', 'Cur_duplicated_0', 'Parid_strip', 'Parid_right', 'Nbhd_right', 'Situs Adrno', 'Situs Adrdir', 'Situs Adrstr', 'Situs Adrsuf', 'Situs Adrsuf2', 'Cityname_right', 'Zoning_right', 'Muni', 'Class_right', 'Luc_right', 'Livunit_right', 'Calcacres_right', 'Location', 'Fronting', 'Street1', 'Street2', 'Traffic', 'Topo1', 'Topo2', 'Topo3', 'Util1', 'Util2', 'Util3', 'Parkprox', 'Parkquanit', 'Parktype', 'Note1', 'Note2', 'Note3', 'Note4', 'Notecd1_right', 'Notecd2_right', 'Ofcard', 'Bldgros D', 'Bldgros V', 'Mscbld N', 'Mscbld V', 'Chgrsn_right', 'Taxdist_right', 'Own1',

In [285]:
for index, df in enumerate(yr_dfs):
    print(yr_dfs[index])
    yr_dfs[index] = yr_dfs[index].select(
        pl.col("Nbhd").cast(pl.Utf8, strict=False),
        pl.col("Luc").cast(pl.Utf8, strict=False),
        pl.col("Aprland").cast(pl.Utf8, strict=False),
        pl.col("Aprbldg").cast(pl.Utf8, strict=False),
        pl.col("Costval").cast(pl.Utf8, strict=False),
        pl.col("Bldgros V").cast(pl.Utf8, strict=False),
        pl.col("Mscbld V").cast(pl.Utf8, strict=False),
        pl.col("*").exclude("Nbhd", "Luc", "Aprland", "Aprbldg", "Costval", "Bldgros V", "Mscbld V")
    )

shape: (68_402, 162)
┌───────┬──────────────┬─────────┬─────────────┬───┬───────┬─────────────┬─────────────┬───────────┐
│ Taxyr ┆ Parid        ┆ Saledt: ┆ Saledt:     ┆ … ┆ D Cdu ┆ Adjfact 2   ┆ D           ┆ Cur_right │
│ ---   ┆ ---          ┆ Year    ┆ Month (Mon) ┆   ┆ ---   ┆ ---         ┆ Pctcomplete ┆ ---       │
│ i16   ┆ str          ┆ (YYYY)  ┆ ---         ┆   ┆ str   ┆ str         ┆ ---         ┆ str       │
│       ┆              ┆ ---     ┆ str         ┆   ┆       ┆             ┆ str         ┆           │
│       ┆              ┆ i64     ┆             ┆   ┆       ┆             ┆             ┆           │
╞═══════╪══════════════╪═════════╪═════════════╪═══╪═══════╪═════════════╪═════════════╪═══════════╡
│ 2011  ┆ 06 0310      ┆ 2010    ┆ Oct         ┆ … ┆ null  ┆ null        ┆ null        ┆ Y         │
│       ┆ LL0417       ┆         ┆             ┆   ┆       ┆             ┆             ┆           │
│ 2011  ┆ 06 0310      ┆ 2010    ┆ Jun         ┆ … ┆ EX    ┆ 1        

In [286]:
sales_parcel_full = pl.concat(yr_dfs, how='diagonal')
print("")
print("Size of complete merged: ", sales_parcel_full.select(pl.count()).item())
print("Total additional rows generated: ", sum_additional_rows)


Size of complete merged:  590887
Total additional rows generated:  -172407
